In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

import os
import time
import datetime
#import data_helpers
#from text_cnn import TextCNN
from tensorflow.contrib import learn
from tensorflow.contrib.learn import preprocessing

/home/kierstenhenderson/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Based on our EDA
max_length = 34

In [3]:
df = pd.read_csv('./data/dataset1_shortjokes_uci_cnn.csv')
#for lowercase 
df.text = df.text.apply(lambda x: x.lower())

In [4]:
df[:5]

,text,funny,not_funny
0,"[me narrating a documentary about narrators] ""...",1,0
1,telling my daughter garlic is good for you. go...,1,0
2,i've been going through a really rough period ...,1,0
3,"if i could have dinner with anyone, dead or al...",1,0
4,two guys walk into a bar. the third guy ducks.,1,0


In [5]:
np.random.seed(6)
shuffle = np.random.permutation(np.arange(df.shape[0]))

X = df['text']
Y_1 = df['funny']
Y_2 = df['not_funny']      

In [6]:
X, Y_1, Y_2 = X[shuffle], Y_1[shuffle],Y_2[shuffle]
#Y = Y[shuffle]


In [7]:
Y = pd.DataFrame({"funny": Y_1.values,"not_funny": Y_2.values})

In [8]:
Y.head()

,funny,not_funny
0,1,0
1,1,0
2,1,0
3,1,0
4,0,1


In [9]:
#X.head()

In [10]:
print('data shape: ', X.shape)
print('label shape:', Y.shape)


test_data, test_label = X[round(X.shape[0]*.9):], Y[round(Y.shape[0]*.9):]
dev_data, dev_label = X[round(X.shape[0]*.7):round(X.shape[0]*.9)], Y[round(Y.shape[0]*.7):round(Y.shape[0]*.9)]
train_data, train_label = X[:round(X.shape[0]*.7)], Y[:round(Y.shape[0]*.7)]


print('test_data', test_data.shape)
print('dev_data', dev_data.shape)
print('train_data', train_data.shape)

data shape:  (463314,)
label shape: (463314, 2)
test_data (46331,)
dev_data (92663,)
train_data (324320,)


In [11]:
max_doc_len = 34
# Setup vocabulary processor
processor = preprocessing.VocabularyProcessor(max_doc_len)

#vocab_processor = tflearn.data_utils.VocabularyProcessor(max_length, min_frequency=0)


processor.fit(train_data)
train = processor.transform(train_data)
dev = processor.transform(dev_data)
test = processor.transform(test_data)

V = len(processor.vocabulary_)
print("unique vocab count:", V)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
unique vocab count: 98302


In [12]:
# Inputs need to be np.arrays instead of generators
x_train = np.array(list(train))
print(x_train.shape)

x_dev = np.array(list(dev))

x_test = np.array(list(test))

y_train = np.array(train_label).astype(int)
y_dev = np.array(dev_label).astype(int)
y_test = np.array(test_label).astype(int)

print(y_train.shape)

(324320, 34)
(324320, 2)


In [13]:
print('corresponding ids\n',x_train[0])
print('corresponding ids\n',y_train[0])

print('corresponding ids\n',x_train[1])
print('corresponding ids\n',x_train[2])
print('corresponding ids\n',x_train[2000])
train_data.iloc[0]
train_data.iloc[0]

corresponding ids
 [1 2 3 4 1 5 6 7 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
corresponding ids
 [1 0]
corresponding ids
 [ 9 10 11 12 13 14 15 16 17  4 17 18 19  4 17 20 17  4 19 21 19  4 19  0
  0  0  0  0  0  0  0  0  0  0]
corresponding ids
 [22 23 24 25  4 26 27  1 28  4 29 30 31 32 33 29 34 23 35 36 24 37  0  0
  0  0  0  0  0  0  0  0  0  0]
corresponding ids
 [  45   31   42 1890 2644  396   42  519 6852 6853    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


'the worst journey in the world by helen back'

In [14]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                Wname = "w_%d"%filter_size
                #W = tf.get_variable(Wname, shape = filter_shape, initializer = tf.contrib.layers.xavier_initializer())
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                
                b = tf.Variable(tf.constant(0.0, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        
        #the line below differs from the tutorial, but is necessary based on changes to Tensorflow
        self.h_pool = tf.concat(pooled_outputs, 3)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.0, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            print("scores shape:", self.scores.shape)
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            
            #correct_predictions = tf.equal(tf.cast(self.predictions,tf.float32), self.input_y)
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))

            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
        
        with tf.name_scope('train'):
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

In [15]:
def batch_generator(ids, labels, batch_size=100):
            
    n_batches = len(ids)//batch_size
    ids, labels = ids[:n_batches*batch_size], labels[:n_batches*batch_size]
    shuffle = np.random.permutation(np.arange(n_batches*batch_size))
    ids, labels = ids[shuffle], labels[shuffle]

    
    for ii in range(0, len(ids), batch_size):
        yield ids[ii:ii+batch_size], labels[ii:ii+batch_size]
                

## Run CNN without Dropout

In [16]:
#Basic training loop:
embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 100
l2_reg_lambda = 0
learning_rate = 0.01
keep_prob = 1.0
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 10

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        sess.run(tf.global_variables_initializer())
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

completed cnn creation
# batches = 5067
Train epoch 0, average loss 0.263021, average accuracy 0.954824,
		Dev epoch 0, average loss 0.235399, average accuracy 0.960824,
				    Time taken for 0 epochs =  498.50074577331543
Train epoch 1, average loss 0.259959, average accuracy 0.970551,
		Dev epoch 1, average loss 0.275175, average accuracy 0.962768,
Train epoch 2, average loss 0.262379, average accuracy 0.975411,
		Dev epoch 2, average loss 0.443942, average accuracy 0.963815,
Train epoch 3, average loss 0.26728, average accuracy 0.977591,
		Dev epoch 3, average loss 0.885046, average accuracy 0.964582,
Train epoch 4, average loss 0.279552, average accuracy 0.977958,
		Dev epoch 4, average loss 1.28894, average accur

## CNN with Dropout 0.5, 256 filters, learning rate 0.01

In [40]:
#Basic training loop:
embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 256
l2_reg_lambda = 0
learning_rate = 0.01
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 10

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        sess.run(tf.global_variables_initializer())
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

completed cnn creation
# batches = 5067
Train epoch 0, average loss 1.07881, average accuracy 0.934925,
		Dev epoch 0, average loss 0.870611, average accuracy 0.954615,
				    Time taken for 0 epochs =  652.840256690979
Train epoch 1, average loss 1.0042, average accuracy 0.940852,
		Dev epoch 1, average loss 0.986357, average accuracy 0.945199,
Train epoch 2, average loss 1.0126, average accuracy 0.932924,
		Dev epoch 2, average loss 1.68322, average accuracy 0.928408,


KeyboardInterrupt: 

## Decrease learning rate to mitigate Overfitting (0.01 to 0.001)


In [14]:
#Actual training loop:
embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 256
l2_reg_lambda = 0
learning_rate = 0.001
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 10

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        sess.run(tf.global_variables_initializer())
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

completed cnn creation
# batches = 5067
Train epoch 0, average loss 0.169832, average accuracy 0.951321,
		Dev epoch 0, average loss 0.0672177, average accuracy 0.97808,
				    Time taken for 0 epochs =  722.7192435264587
Train epoch 1, average loss 0.0586678, average accuracy 0.982466,
		Dev epoch 1, average loss 0.0609224, average accuracy 0.983036,
Train epoch 2, average loss 0.0375557, average accuracy 0.989608,
		Dev epoch 2, average loss 0.078718, average accuracy 0.982917,
Train epoch 3, average loss 0.0280478, average accuracy 0.993413,
		Dev epoch 3, average loss 0.109895, average accuracy 0.982852,
Train epoch 4, average loss 0.0226164, average accuracy 0.995236,
		Dev epoch 4, average loss 0.152929, average

## Decrease learning rate again to mitigate Overfitting (0.001 to 0.0001)
- ran 12/1

In [41]:
#Actual training loop:
embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 256
l2_reg_lambda = 0
learning_rate = 0.0001
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 15

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        sess.run(tf.global_variables_initializer())
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
completed cnn creation
# batches = 5067
Train epoch 0, average loss 0.50275, average accuracy 0.876388,
		Dev epoch 0, average loss 0.129351, average accuracy 0.954172,
				    Time taken for 0 epochs =  652.1811921596527
Train epoch 1, average loss 0.152941, average accuracy 0.943933,
		Dev epoch 1, average loss 0.0981918, average accuracy 0.964312,
Train epoch 2, average loss 0.102196, average accuracy 0.962453,
		Dev epoch 2, average loss 0.0870992, average accuracy 0.967875,
Train epoch 3, average loss 0.0753122, average accuracy 0.972333,
		Dev epoch 3, average loss 0.0673719, average accuracy 0.975153,
Train epoch 4, average loss 0.058899, average accuracy 0.978528,
		Dev epoch 4, average loss 0.0626418, average accuracy 0.977075,
Train epoch 5, average loss 0.0478803, average accuracy 0.982697,
		Dev epoch 5, average loss 0.0576026, average accuracy 0.978674,
Train epoch 6, average loss 0.0384907, av

KeyboardInterrupt: 

## Trying smaller filter size

In [14]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                Wname = "w_%d"%filter_size
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                
                b = tf.Variable(tf.constant(0.0, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        
        #the line below differs from the tutorial, but is necessary based on changes to Tensorflow
        self.h_pool = tf.concat(pooled_outputs, 3)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.0, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            print("scores shape:", self.scores.shape)
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            
            #correct_predictions = tf.equal(tf.cast(self.predictions,tf.float32), self.input_y)
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))

            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
            
        with tf.name_scope('train'):
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)


In [15]:
def batch_generator(ids, labels, batch_size=100):
            
    n_batches = len(ids)//batch_size
    ids, labels = ids[:n_batches*batch_size], labels[:n_batches*batch_size]
    shuffle = np.random.permutation(np.arange(n_batches*batch_size))
    ids, labels = ids[shuffle], labels[shuffle]

    
    for ii in range(0, len(ids), batch_size):
        yield ids[ii:ii+batch_size], labels[ii:ii+batch_size]
            

In [16]:
#Actual training loop:
embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 100
l2_reg_lambda = 0
learning_rate = 0.01
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 10

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        sess.run(tf.global_variables_initializer())
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

completed cnn creation
# batches = 5067
Train epoch 0, average loss 0.48908, average accuracy 0.934222,
		Dev epoch 0, average loss 0.40466, average accuracy 0.952218,
				    Time taken for 0 epochs =  529.6132485866547
Train epoch 1, average loss 0.579214, average accuracy 0.942159,
		Dev epoch 1, average loss 0.528759, average accuracy 0.949918,
Train epoch 2, average loss 0.598762, average accuracy 0.93853,
		Dev epoch 2, average loss 0.810308, average accuracy 0.947499,
Train epoch 3, average loss 0.616179, average accuracy 0.937016,
		Dev epoch 3, average loss 1.45503, average accuracy 0.942305,
Train epoch 4, average loss 0.594433, average accuracy 0.934919,
		Dev epoch 4, average loss 1.85442, average accuracy 

KeyboardInterrupt: 

## Filter size 100, learning rate 0.0001

In [20]:
#Actual training loop:
embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 100
l2_reg_lambda = 0
learning_rate = 0.0001
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 15

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        sess.run(tf.global_variables_initializer())
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        
        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            
            #save_path = saver.save(sess, "./runs/cnn/checkpoints/my_training_final_model.ckpt")

            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
completed cnn creation
# batches = 5067
Train epoch 0, average loss 0.549724, average accuracy 0.854965,
		Dev epoch 0, average loss 0.145365, average accuracy 0.948752,
				    Time taken for 0 epochs =  482.00134205818176
Train epoch 1, average loss 0.167735, average accuracy 0.937562,
		Dev epoch 1, average loss 0.108812, average accuracy 0.960921,
Train epoch 2, average loss 0.114123, average accuracy 0.958065,
		Dev epoch 2, average loss 0.0856902, average accuracy 0.969301,
Train epoch 3, average loss 0.0859665, average accuracy 0.968284,
		Dev epoch 3, average loss 0.0727051, average accuracy 0.97362,
Train epoch 4, average loss 0.0683465, average accuracy 0.975016,
		Dev epoch 4, average loss 0.0644522, average accuracy 0.976654,
Train epoch 5, average loss 0.0554454, average accuracy 0.979827,
		Dev epoch 5, average loss 0.0598681, average accuracy 0.978285,
Train epoch 6, average loss 0.0459915, a

## Saving model created with 100 filters, learning rate 0.0001 as: my_training_final_model_2

In [21]:
embed_dim = 300 
filter_sizes= [3,4,5]
num_filters = 100
l2_reg_lambda = 0
learning_rate = 0.0001
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 10

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))



with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        
        sess.run(tf.global_variables_initializer())
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        
        saver = tf.train.Saver()

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))
            save_path = saver.save(sess, "./runs/cnn/checkpoints/my_training_final_model_2.ckpt")
            
            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                
                
            
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
completed cnn creation
# batches = 5067
Train epoch 0, average loss 0.523222, average accuracy 0.859162,
		Dev epoch 0, average loss 0.142759, average accuracy 0.94941,
				    Time taken for 0 epochs =  484.31138467788696
Train epoch 1, average loss 0.164819, average accuracy 0.938863,
		Dev epoch 1, average loss 0.1079, average accuracy 0.961396,
Train epoch 2, average loss 0.112586, average accuracy 0.958716,
		Dev epoch 2, average loss 0.08599, average accuracy 0.969171,
Train epoch 3, average loss 0.0857723, average accuracy 0.968716,
		Dev epoch 3, average loss 0.0737298, average accuracy 0.973296,
Train epoch 4, average loss 0.0681073, average accuracy 0.974754,
		Dev epoch 4, average loss 0.0657978, average accuracy 0.976179,
Train epoch 5, average loss 0.0564885, average accuracy 0.97946,
		Dev epoch 5, average loss 0.0607796, average accuracy 0.97781,
Train epoch 6, average loss 0.0467574, average

In [50]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto()
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("./runs/cnn/checkpoints/my_training_final_model_2.ckpt.meta")
        saver.restore(sess, "./runs/cnn/checkpoints/my_training_final_model_2.ckpt")
        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        
        test_predictions = sess.run(predictions, {input_x: x_test, dropout_keep_prob: 1.0})

INFO:tensorflow:Restoring parameters from ./runs/cnn/checkpoints/my_training_final_model_2.ckpt


In [51]:
len(test_predictions)

46331

In [52]:
type(test_predictions)

numpy.ndarray

In [53]:
test_predictions[:5]

array([1, 1, 0, 1, 0])

In [54]:
actual_class_test = y_test[:,1]
actual_class_test[:5]

array([1, 1, 0, 1, 0])

In [55]:
correct_predictions = float(sum(test_predictions == actual_class_test))
print("Total number of test examples: {}".format(len(actual_class_test)))
print("Accuracy: {:g}".format(correct_predictions/float(len(actual_class_test))))

Total number of test examples: 46331
Accuracy: 0.981978


In [56]:
text = pd.DataFrame({"text_example": test_data})

In [57]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

In [58]:
text[:5]

,text_example
452670,spring arrives thursday afternoon
257774,older women are most vulnerable to cervical cancer
8829,why did the run-on sentence take a pregnancy test? because its period came too late.
411172,tv review: 'rosemary's baby'
153671,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo


In [59]:
text = text.reset_index(drop=True)
text[:5]

,text_example
0,spring arrives thursday afternoon
1,older women are most vulnerable to cervical cancer
2,why did the run-on sentence take a pregnancy test? because its period came too late.
3,tv review: 'rosemary's baby'
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo


In [60]:
test_label[:5]

,funny,not_funny
0,0,1
1,0,1
2,1,0
3,0,1
4,1,0


In [61]:
test_label = test_label.reset_index(drop=True)

In [62]:
test_label[:5]

,funny,not_funny
0,0,1
1,0,1
2,1,0
3,0,1
4,1,0


In [63]:
df = text.merge(test_label, left_index=True, right_index=True)

In [64]:
df[:5]

,text_example,funny,not_funny
0,spring arrives thursday afternoon,0,1
1,older women are most vulnerable to cervical cancer,0,1
2,why did the run-on sentence take a pregnancy test? because its period came too late.,1,0
3,tv review: 'rosemary's baby',0,1
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo,1,0


In [65]:
test_predictions = pd.Series(test_predictions)

In [66]:
df_test = df.merge(test_predictions.to_frame(), left_index=True, right_index=True)

In [67]:
df_test[:5]

,text_example,funny,not_funny,0
0,spring arrives thursday afternoon,0,1,1
1,older women are most vulnerable to cervical cancer,0,1,1
2,why did the run-on sentence take a pregnancy test? because its period came too late.,1,0,0
3,tv review: 'rosemary's baby',0,1,1
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo,1,0,0


In [68]:
df_test.columns = ['text', 'funny', 'not_funny', 'predicted_class']

In [69]:
df_test[:5]

,text,funny,not_funny,predicted_class
0,spring arrives thursday afternoon,0,1,1
1,older women are most vulnerable to cervical cancer,0,1,1
2,why did the run-on sentence take a pregnancy test? because its period came too late.,1,0,0
3,tv review: 'rosemary's baby',0,1,1
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo,1,0,0


In [70]:
df_test['correct_prediction'] = df_test.not_funny == df_test.predicted_class

In [71]:
df_test[:5]

,text,funny,not_funny,predicted_class,correct_prediction
0,spring arrives thursday afternoon,0,1,1,True
1,older women are most vulnerable to cervical cancer,0,1,1,True
2,why did the run-on sentence take a pregnancy test? because its period came too late.,1,0,0,True
3,tv review: 'rosemary's baby',0,1,1,True
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo,1,0,0,True


In [72]:
df_test.correct_prediction.mean()

0.9819775096587597

In [74]:
#df_test.to_csv("./evaluating_model_results/base_cnn_100filters_0.0001LR.csv")

## Saving model created with 256 filters, Learning Rate 0.0001 as my_training_final_model_3

In [75]:
embed_dim = 300 #use when not using pre-trained embeddings
filter_sizes= [3,4,5]
num_filters = 256
l2_reg_lambda = 0
learning_rate = 0.0001
keep_prob = 0.5
evaluate_train = 1 # of epochs at which to print test accuracy
evaluate_dev = 1 # of epochs at which to estimate and print dev accuracy
time_print = 12 # of epochs at which to print time taken
num_classes = 2
num_epochs = 8

num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))



with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, embedding_size=embed_dim, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
       
        sess.run(tf.global_variables_initializer())
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        
        saver = tf.train.Saver()

        #saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        # Write vocabulary
        ## vocab_processor.save(os.path.join(out_dir, "vocab"))

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))
            save_path = saver.save(sess, "./runs/cnn/checkpoints/my_training_final_model_3.ckpt")
            
            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                num_batches = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                    num_batches += 1
                avg_loss = total_loss/(num_batches*batch_size)
                avg_acc = total_acc/(num_batches*batch_size)
                #time_str = datetime.datetime.now().isoformat()
                print("\t\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))    
            
            if e%time_print == 0:
                end = time.time()
                print("\t\t\t\t    Time taken for",e,"epochs = ", end-start)

Writing to /home/kierstenhenderson/Final_project/runs/cnn

scores shape: (?, 2)
completed cnn creation
# batches = 5067
Train epoch 0, average loss 0.460008, average accuracy 0.880372,
		Dev epoch 0, average loss 0.131682, average accuracy 0.954,
				    Time taken for 0 epochs =  618.1741096973419
Train epoch 1, average loss 0.148801, average accuracy 0.945339,
		Dev epoch 1, average loss 0.0972129, average accuracy 0.964873,
Train epoch 2, average loss 0.100837, average accuracy 0.963273,
		Dev epoch 2, average loss 0.0782043, average accuracy 0.971935,
Train epoch 3, average loss 0.0756959, average accuracy 0.972429,
		Dev epoch 3, average loss 0.0684272, average accuracy 0.975369,
Train epoch 4, average loss 0.0592153, average accuracy 0.978331,
		Dev epoch 4, average loss 0.061996, average accuracy 0.977961,
Train epoch 5, average loss 0.0472042, average accuracy 0.982778,
		Dev epoch 5, average loss 0.0583225, average accuracy 0.979678,
Train epoch 6, average loss 0.0379949, aver

In [76]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto()
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("./runs/cnn/checkpoints/my_training_final_model_3.ckpt.meta")
        saver.restore(sess, "./runs/cnn/checkpoints/my_training_final_model_3.ckpt")
        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        
        test_predictions = sess.run(predictions, {input_x: x_test, dropout_keep_prob: 1.0})

INFO:tensorflow:Restoring parameters from ./runs/cnn/checkpoints/my_training_final_model_3.ckpt


In [77]:
test_predictions[:5]

array([1, 1, 0, 1, 0])

In [78]:
actual_class_test = y_test[:,1]
actual_class_test[:5]

array([1, 1, 0, 1, 0])

In [79]:
correct_predictions = float(sum(test_predictions == actual_class_test))
print("Total number of test examples: {}".format(len(actual_class_test)))
print("Accuracy: {:g}".format(correct_predictions/float(len(actual_class_test))))

Total number of test examples: 46331
Accuracy: 0.982021


In [80]:
text = pd.DataFrame({"text_example": test_data})

In [81]:
text = text.reset_index(drop=True)
text[:5]

,text_example
0,spring arrives thursday afternoon
1,older women are most vulnerable to cervical cancer
2,why did the run-on sentence take a pregnancy test? because its period came too late.
3,tv review: 'rosemary's baby'
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo


In [82]:
test_label = test_label.reset_index(drop=True)

In [83]:
test_label[:5]

,funny,not_funny
0,0,1
1,0,1
2,1,0
3,0,1
4,1,0


In [84]:
df = text.merge(test_label, left_index=True, right_index=True)

In [85]:
df[:5]

,text_example,funny,not_funny
0,spring arrives thursday afternoon,0,1
1,older women are most vulnerable to cervical cancer,0,1
2,why did the run-on sentence take a pregnancy test? because its period came too late.,1,0
3,tv review: 'rosemary's baby',0,1
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo,1,0


In [86]:
test_predictions = pd.Series(test_predictions)

In [87]:
df_test = df.merge(test_predictions.to_frame(), left_index=True, right_index=True)

In [88]:
df_test[:5]

,text_example,funny,not_funny,0
0,spring arrives thursday afternoon,0,1,1
1,older women are most vulnerable to cervical cancer,0,1,1
2,why did the run-on sentence take a pregnancy test? because its period came too late.,1,0,0
3,tv review: 'rosemary's baby',0,1,1
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo,1,0,0


In [89]:
df_test.columns = ['text', 'funny', 'not_funny', 'predicted_class']

In [90]:
df_test[:5]

,text,funny,not_funny,predicted_class
0,spring arrives thursday afternoon,0,1,1
1,older women are most vulnerable to cervical cancer,0,1,1
2,why did the run-on sentence take a pregnancy test? because its period came too late.,1,0,0
3,tv review: 'rosemary's baby',0,1,1
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo,1,0,0


In [91]:
df_test['correct_prediction'] = df_test.not_funny == df_test.predicted_class

In [92]:
df_test[:5]

,text,funny,not_funny,predicted_class,correct_prediction
0,spring arrives thursday afternoon,0,1,1,True
1,older women are most vulnerable to cervical cancer,0,1,1,True
2,why did the run-on sentence take a pregnancy test? because its period came too late.,1,0,0,True
3,tv review: 'rosemary's baby',0,1,1,True
4,i was at the natural history museum and i saw the neanderthal exhibit. those guys were buff studs. no homo,1,0,0,True


In [93]:
df_test.correct_prediction.mean()

0.9820206773002957

In [94]:
df_test.to_csv("./evaluating_model_results/base_cnn_256filters_0.0001LR.csv")

## Scoring Dataset2 with model made with Dataset 1: 

## First applied model with 100 filters, it worked better than the one created with 256 filters so we used it to perform our Analysis of Dataset 2


In [29]:
df = pd.read_csv("./data/dataset2_crowdtruth_working.csv")

In [30]:
df[:5]

,text,funny,not_funny
0,"left luggage , also luggage storage or bag sto...",0,1
1,their name comes from the title of the the fla...,0,1
2,the lyside sulphur can grow from egg to adult ...,0,1
3,istanbul suicide bomber entered turkey as a m...,0,1
4,"both describe god as the one divine person , j...",0,1


In [31]:
X = df['text']
Y_1 = df['funny']
Y_2 = df['not_funny']

In [32]:
len(X)

10868

In [33]:
Y = pd.DataFrame({"funny": Y_1.values,"not_funny": Y_2.values})

In [34]:
Y[:5]

,funny,not_funny
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [35]:
dataset2_data, dataset2_label = X, Y
print('dataset2_data', dataset2_data.shape)

dataset2_data (10868,)


In [36]:
dataset2 = processor.transform(dataset2_data)

In [37]:
x_dataset2 = np.array(list(dataset2))
y_dataset2 = np.array(dataset2_label).astype(int)

print(x_dataset2.shape)
print(y_dataset2.shape)

(10868, 34)
(10868, 2)


## Dataset 2 predictions with model to test scoring

In [38]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto()
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("./runs/cnn/checkpoints/my_training_final_model_2.ckpt.meta")
        saver.restore(sess, "./runs/cnn/checkpoints/my_training_final_model_2.ckpt")
        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        
        dataset2_predictions = sess.run(predictions, {input_x: x_dataset2, dropout_keep_prob: 1.0})

INFO:tensorflow:Restoring parameters from ./runs/cnn/checkpoints/my_training_final_model_2.ckpt


In [39]:
dataset2_predictions[:5]

array([0, 0, 1, 0, 0])

In [40]:
y_dataset2[:5]
#y_dataset2 = np.array(dataset2_label).astype(int)

correct_scores_dataset2 = np.array(Y_2).astype(int)

In [41]:
correct_predictions = float(sum(dataset2_predictions == correct_scores_dataset2))
print("Total number of test examples: {}".format(len(correct_scores_dataset2)))
print("Accuracy: {:g}".format(correct_predictions/float(len(correct_scores_dataset2))))

Total number of test examples: 10868
Accuracy: 0.785793


In [42]:
dataset2_predictions_series = pd.Series(dataset2_predictions)

In [43]:
df = df.merge(dataset2_predictions_series.to_frame(), left_index=True, right_index=True)

In [44]:
df[:5]

,text,funny,not_funny,0
0,"left luggage , also luggage storage or bag sto...",0,1,0
1,their name comes from the title of the the fla...,0,1,0
2,the lyside sulphur can grow from egg to adult ...,0,1,1
3,istanbul suicide bomber entered turkey as a m...,0,1,0
4,"both describe god as the one divine person , j...",0,1,0


In [45]:
df.columns = ['text', 'funny', 'not_funny', 'predicted_class']

In [46]:
df['correct_prediction'] = df.not_funny == df.predicted_class

In [47]:
df[:5]

,text,funny,not_funny,predicted_class,correct_prediction
0,"left luggage , also luggage storage or bag sto...",0,1,0,False
1,their name comes from the title of the the fla...,0,1,0,False
2,the lyside sulphur can grow from egg to adult ...,0,1,1,True
3,istanbul suicide bomber entered turkey as a m...,0,1,0,False
4,"both describe god as the one divine person , j...",0,1,0,False


In [50]:
df.correct_prediction.mean()

0.7857931542142068

In [51]:
#df.to_csv("./evaluating_model_results/dataset2_predictions_base_cnn_100_filters_0.0001LR.csv")

## Dataset 2  with Replacement Predictions (least common word with its most common synonym) 

In [52]:
df = pd.read_csv("./data/dataset2_crowdtruth_working_single_word_replacement.csv")

In [53]:
X = df['text']
Y_1 = df['funny']
Y_2 = df['not_funny']

In [54]:
Y = pd.DataFrame({"funny": Y_1.values,"not_funny": Y_2.values})

In [55]:
dataset2_data_replacement, dataset2_label_replacement = X, Y
print('dataset2_data_replacement', dataset2_data_replacement.shape)

dataset2_data_replacement (10868,)


In [56]:
dataset2 = processor.transform(dataset2_data_replacement)

In [57]:
x_dataset2_replacement = np.array(list(dataset2))
y_dataset2_replacement = np.array(dataset2_label_replacement).astype(int)

print(x_dataset2_replacement.shape)
print(y_dataset2_replacement.shape)

(10868, 34)
(10868, 2)


In [58]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto()
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("./runs/cnn/checkpoints/my_training_final_model_2.ckpt.meta")
        saver.restore(sess, "./runs/cnn/checkpoints/my_training_final_model_2.ckpt")
        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        
        dataset2_predict_replacements = sess.run(predictions, {input_x: x_dataset2_replacement, dropout_keep_prob: 1.0})

INFO:tensorflow:Restoring parameters from ./runs/cnn/checkpoints/my_training_final_model_2.ckpt


In [59]:
dataset2_predict_replacements[:5]

array([0, 0, 1, 0, 0])

In [60]:
actual_class_replacements = y_dataset2_replacement[:,1]
actual_class_replacements[:5]

array([1, 1, 1, 1, 1])

In [61]:
dataset2_predictions_series = pd.Series(dataset2_predict_replacements)

In [62]:
dataset2_predictions_series[:5]

0    0
1    0
2    1
3    0
4    0
dtype: int64

In [63]:
df = df.merge(dataset2_predictions_series.to_frame(), left_index=True, right_index=True)

In [64]:
df[:5]

,text,funny,not_funny,0
0,"left luggage , also luggage storage or bag sto...",0,1,0
1,their name comes from the title of the the fla...,0,1,0
2,the lyside sulphur can grow from egg to adult ...,0,1,1
3,istanbul suicide bomber entered turkey as a m...,0,1,0
4,"both describe god as the one divine person , j...",0,1,0


In [65]:
df.columns = ['text', 'funny', 'not_funny', 'predicted_class']

In [66]:
df['correct_prediction'] = df.not_funny == df.predicted_class

In [67]:
df[:5]

,text,funny,not_funny,predicted_class,correct_prediction
0,"left luggage , also luggage storage or bag sto...",0,1,0,False
1,their name comes from the title of the the fla...,0,1,0,False
2,the lyside sulphur can grow from egg to adult ...,0,1,1,True
3,istanbul suicide bomber entered turkey as a m...,0,1,0,False
4,"both describe god as the one divine person , j...",0,1,0,False


In [68]:
df.correct_prediction.mean()

0.7857931542142068

In [69]:
#df.to_csv("./evaluating_model_results/dataset2_one_replacement_predictions_base_cnn_100_filters_0.0001LR.csv")

## Dataset 2 with Replacement Predictions (all possible words with most common synonym)

In [70]:
df = pd.read_csv("./data/dataset2_crowdtruth_working_all_replacement.csv")

In [72]:
X = df['text']
Y_1 = df['funny']
Y_2 = df['not_funny']

In [73]:
Y = pd.DataFrame({"funny": Y_1.values,"not_funny": Y_2.values})

In [74]:
dataset2_data_replacement, dataset2_label_replacement = X, Y
print('dataset2_data_replacement', dataset2_data_replacement.shape)

dataset2_data_replacement (10868,)


In [75]:
dataset2 = processor.transform(dataset2_data_replacement)

In [76]:
x_dataset2_replacement = np.array(list(dataset2))
y_dataset2_replacement = np.array(dataset2_label_replacement).astype(int)

print(x_dataset2_replacement.shape)
print(y_dataset2_replacement.shape)

(10868, 34)
(10868, 2)


In [77]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto()
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("./runs/cnn/checkpoints/my_training_final_model_2.ckpt.meta")
        saver.restore(sess, "./runs/cnn/checkpoints/my_training_final_model_2.ckpt")
        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        
        dataset2_predict_replacements = sess.run(predictions, {input_x: x_dataset2_replacement, dropout_keep_prob: 1.0})

INFO:tensorflow:Restoring parameters from ./runs/cnn/checkpoints/my_training_final_model_2.ckpt


In [78]:
dataset2_predict_replacements[:5]

array([0, 0, 0, 0, 0])

In [79]:
actual_class_replacements = y_dataset2_replacement[:,1]
actual_class_replacements[:5]

array([0, 0, 0, 0, 0])

In [80]:
dataset2_predictions_series = pd.Series(dataset2_predict_replacements)

In [81]:
dataset2_predictions_series[:5]

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [82]:
df = df.merge(dataset2_predictions_series.to_frame(), left_index=True, right_index=True)

In [83]:
df.columns = ['text', 'funny', 'not_funny', 'predicted_class']

In [84]:
df[:5]

,text,funny,not_funny,predicted_class
0,dont’ belt on death’s door. make the bell and ...,1,0,0
1,1 see 1 call the like way whether im’ about to...,1,0,0
2,my wife use to be a even customer at mcdonalds...,1,0,0
3,"2 can of key have married, after the bride whi...",1,0,0
4,"- dont' be irreplaceable, if you bank' be exch...",1,0,0


In [85]:
df['correct_prediction'] = df.not_funny == df.predicted_class

In [86]:
df.correct_prediction.mean()

0.7513801987486198

In [87]:
df[:5]

,text,funny,not_funny,predicted_class,correct_prediction
0,dont’ belt on death’s door. make the bell and ...,1,0,0,True
1,1 see 1 call the like way whether im’ about to...,1,0,0,True
2,my wife use to be a even customer at mcdonalds...,1,0,0,True
3,"2 can of key have married, after the bride whi...",1,0,0,True
4,"- dont' be irreplaceable, if you bank' be exch...",1,0,0,True


In [89]:
#df.to_csv("./evaluating_model_results/dataset2_preds_all_replacements_base_cnn_100_filters_0.0001LR.csv")